# import libraries

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import sklearn
from sklearn import metrics
import xgboost as xgb
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import joblib
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, cohen_kappa_score, confusion_matrix
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

# read the data

In [2]:
df = pd.read_csv("PharmaHacks2022-master/Samplenormalized_train_data.csv", index_col=False)
X = df.iloc[:, 2:1096]
Y = df.iloc[:, 1096]

# use weighting to provide higher weight to class with fewer samples

In [3]:
w = [len(Y) / len(Y[Y == 'Healthy']), len(Y) / len(Y[Y == 'Disease-1']), len(Y) / len(Y[Y == 'Disease-2']),
                                                   len(Y) / len(Y[Y == 'Disease-3'])]

# assign labels to classes

In [4]:
Y[Y == 'Healthy'] = 0
Y[Y == 'Disease-1'] = 1
Y[Y == 'Disease-2'] = 2
Y[Y == 'Disease-3'] = 3

# prepare data for xgboost

In [5]:
import xgboost as xgb
#create data matrix
data_matrix = xgb.DMatrix(data = X, label = Y)

# create train and validation split
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size = 0.2, random_state = 0)
# the weight for each class needs to be assigned to the samples individually in xgboost. So we create a weight matrix
#that is the same length as samples in the train data
convert_to_weight = {'0': w[0], '1': w[1], '2':w[2]*10, '3': w[3]}
weight_matrix = [convert_to_weight[str(i)] for i in y_train]

# hyperparameter tuning

In [6]:
# params = {
#         'n_estimators': [50, 100, 150, 200],
#         'max_depth': [6, 8, 10],
#         'learning_rate': [0.05, 0.1, 0.2],
#         'gamma': [0, 0.5, 1],
#         'min_child_weight': [1, 5, 10],
#         'subsample': [0.6, 0.8, 1],
#         'colsample_by_tree': [0.6, 0.8, 1]}

# xgb = XGBClassifier(objective='multi:softprob', silent=True, nthread=6)
# evaluation = [( X_train, y_train), ( X_val, y_val)]

# grid_search = GridSearchCV(estimator = xgb, param_grid=params, verbose=3, scoring = 'f1_weighted')
# grid_search.fit(X_train, y_train, eval_set=evaluation, eval_metric="mlogloss", early_stopping_rounds=10, sample_weight = weight_matrix)

Point to be noted, we could not select the complete hyperparameter space we wanted to explore. Therefore, we limited it to manual search of the features we thought was significant enough and seleced the best performing one. 

# hyperparameter selected

In [ ]:
#selected parameters
params = {
            'objective':'multi:softprob',
            'max_depth': 8,
            'learning_rate': 0.1,
            'n_estimators':200,
            'n_threads' : 8,
            'colsample_bytree': 0.8,
            'alpha': 10
        }
           
# initialize the classifier 
xgb_clf = XGBClassifier(**params)

#initialize data
evaluation = [( X_train, y_train), ( X_val, y_val)]

#fit the model
xgb_clf.fit(X_train, y_train, eval_set=evaluation, eval_metric="mlogloss", early_stopping_rounds=100, sample_weight = weight_matrix)

[07:04:28] WARNING: ../src/learner.cc:576: 
Parameters: { "n_threads" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-mlogloss:1.34363	validation_1-mlogloss:1.35466
[1]	validation_0-mlogloss:1.30029	validation_1-mlogloss:1.32114
[2]	validation_0-mlogloss:1.26805	validation_1-mlogloss:1.30200
[3]	validation_0-mlogloss:1.23843	validation_1-mlogloss:1.28154
[4]	validation_0-mlogloss:1.20992	validation_1-mlogloss:1.26197
[5]	validation_0-mlogloss:1.19068	validation_1-mlogloss:1.25127
[6]	validation_0-mlogloss:1.16615	validation_1-mlogloss:1.23641
[7]	validation_0-mlogloss:1.14353	validation_1-mlogloss:1.22386
[8]	validation_0-mlogloss:1.12153	validation_1-mlogloss:1.21046
[9]	validation_0-mlogloss:1.10092	validation_1-mlogloss:1.19927


[121]	validation_0-mlogloss:0.08581	validation_1-mlogloss:0.52671
[122]	validation_0-mlogloss:0.08481	validation_1-mlogloss:0.52575
[123]	validation_0-mlogloss:0.08372	validation_1-mlogloss:0.52446
[124]	validation_0-mlogloss:0.08281	validation_1-mlogloss:0.52356
[125]	validation_0-mlogloss:0.08186	validation_1-mlogloss:0.52306
[126]	validation_0-mlogloss:0.08092	validation_1-mlogloss:0.52223
[127]	validation_0-mlogloss:0.08011	validation_1-mlogloss:0.52204
[128]	validation_0-mlogloss:0.07926	validation_1-mlogloss:0.52118
[129]	validation_0-mlogloss:0.07844	validation_1-mlogloss:0.52065
[130]	validation_0-mlogloss:0.07776	validation_1-mlogloss:0.52047
[131]	validation_0-mlogloss:0.07701	validation_1-mlogloss:0.51989
[132]	validation_0-mlogloss:0.07624	validation_1-mlogloss:0.51919
[133]	validation_0-mlogloss:0.07543	validation_1-mlogloss:0.51823
[134]	validation_0-mlogloss:0.07469	validation_1-mlogloss:0.51766
[135]	validation_0-mlogloss:0.07398	validation_1-mlogloss:0.51754
[136]	vali

# check performance in validation data

In [ ]:
#make prediction
y_pred = xgb_clf.predict(X_val)

print('weighted f1 score ', f1_score(list(y_val), list(y_pred), average = 'weighted'))
print('per classs f1 score ', f1_score(list(y_val), list(y_pred), average = None))
print('confusion matrix \n', confusion_matrix(list(y_val), list(y_pred)))
print('cohen cappa score ', cohen_kappa_score(list(y_val), list(y_pred)))

# save model 

In [ ]:
joblib.dump(xgb_clf, 'model_final.pkl')

# bacterias which are informative for the prediciton

In [ ]:
feature_important = xgb_clf.get_booster().get_score(importance_type='weight')
keys = list(feature_important.keys())
values = list(feature_important.values())

data = pd.DataFrame(data=values, index=keys, columns=["score"]).sort_values(by = "score", ascending=False)
print(len(data))

443 bacterias were deemed important by the model.

# test data prediction

In [ ]:
#read and process the test data the same way as train data

df = pd.read_csv("PharmaHacks2022-master/Samplenormalized_test_data.csv", index_col=False)
X_test = df.iloc[:, 2:1096]
Y_test = df.iloc[:, 1096]

Y_test[Y_test == 'Healthy'] = 0
Y_test[Y_test == 'Disease-1'] = 1
Y_test[Y_test == 'Disease-2'] = 2
Y_test[Y_test == 'Disease-3'] = 3

In [ ]:
model = joblib.load("model_file_name.pkl")
y_pred = model.predict(X_test)

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import cohen_kappa_score, confusion_matrix

print('weighted f1 score ', f1_score(list(Y_test), list(y_pred), average = 'weighted'))
print('per classs f1 score ', f1_score(list(Y_test), list(y_pred), average = None))
print('confusion matrix \n', confusion_matrix(list(Y_test), list(y_pred)))
print('cohen cappa score ', cohen_kappa_score(list(Y_test), list(y_pred)))

# differentially expressed bacteria

In [ ]:
#reading the csv file created in R. need some string formating as R doesn't read hyphens
diff_bacts = pd.read_csv('DEB_strictfiltered_training.csv')
diff_bacts = list(diff_bacts.iloc[:,0])
diff_bacts = ['Bacteria.' + bact.split('Bacteria.')[1] for bact in diff_bacts]

### compare the differenetially enriched bacteria and bacteria found to be important by our xgboost model

In [ ]:
count = 0
for bact in diff_bacts:
    if bact not in keys:
        count += 1
        print('found a enriched bacteria that is not identified by the model')
print(count)

So all the bacteria found to be important by xgboost are enriched.

# What if we only use the enriched bacteria to train the model (6.5% data)?

Now we see whether we can just use this statistically significant bacteria to train the model. The motivation is that if we find the bacteria which are required to classify between different classes it would reduce our screening space. Also it would reduce the complexity of the model.

In [ ]:
df = pd.read_csv("PharmaHacks2022-master/Samplenormalized_train_data.csv", index_col=False)
X = df[diff_bacts]
Y = df.iloc[:, 1096]

# the weight for each class needs to be assigned to the samples individually in xgboost. So we create a weight matrix
#that is the same length as samples in the train data
convert_to_weight = {'0': w[0], '1': w[1], '2':w[2]*10, '3': w[3]}
weight_matrix = [convert_to_weight[str(i)] for i in y_train]

Y[Y == 'Healthy'] = 0
Y[Y == 'Disease-1'] = 1
Y[Y == 'Disease-2'] = 2
Y[Y == 'Disease-3'] = 3

#create data matrix
import xgboost as xgb
data_matrix = xgb.DMatrix(data = X, label = Y)

# create train and validation split
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size = 0.2, random_state = 0)


#selected parameters
params = {
            'objective':'multi:softprob',
            'max_depth': 8,
            'learning_rate': 0.1,
            'n_estimators':200,
            'n_threads' : 8,
            'colsample_bytree': 0.8,
            'alpha': 10
        }
           
# initialize the classifier 
xgb_clf = XGBClassifier(**params)

#initialize data
evaluation = [( X_train, y_train), ( X_val, y_val)]

#fit the model
xgb_clf.fit(X_train, y_train, eval_set=evaluation, eval_metric="mlogloss", early_stopping_rounds=100, sample_weight = weight_matrix)

#save model
joblib.dump(xgb_clf, 'model_71_bacteria.pkl')

#read test data and process
df = pd.read_csv("PharmaHacks2022-master/Samplenormalized_test_data.csv", index_col=False)
X_test = df[diff_bacts]

Y_test = df.iloc[:, 1096]

Y_test[Y_test == 'Healthy'] = 0
Y_test[Y_test == 'Disease-1'] = 1
Y_test[Y_test == 'Disease-2'] = 2
Y_test[Y_test == 'Disease-3'] = 3

y_pred = xgb_clf.predict(X_test)

In [ ]:
print('weighted f1 score ', f1_score(list(Y_test), list(y_pred), average = 'weighted'))
print('per classs f1 score ', f1_score(list(Y_test), list(y_pred), average = None))
print('confusion matrix \n', confusion_matrix(list(Y_test), list(y_pred)))
print('cohen cappa score ', cohen_kappa_score(list(Y_test), list(y_pred)))

### we have been able to achieve comparable performance using only 71 bacteria using differentially enrichment analysis